Preprocessing Joint Positions, Orientations

In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Code Outline

*   Split data into train/validation dictionaries
*   Loop through each patient, exercise in dataset
*   Read, filter joint position data for 25 joints/body parts
*   Read, normalize three labels
*   Add data to annotations list
*   After loop, add split and annotations to pkl file

In [ ]:
# Define indices for joint positions (same for each JointPosition csv file)
index_Spine_Base = 0
index_Spine_Mid = 4
index_Neck = 8
index_Head = 12  # no orientation
index_Shoulder_Left = 16
index_Elbow_Left = 20
index_Wrist_Left = 24
index_Hand_Left = 28
index_Shoulder_Right = 32
index_Elbow_Right = 36
index_Wrist_Right = 40
index_Hand_Right = 44
index_Hip_Left = 48
index_Knee_Left = 52
index_Ankle_Left = 56
index_Foot_Left = 60  # no orientation
index_Hip_Right = 64
index_Knee_Right = 68
index_Ankle_Right = 72
index_Foot_Right = 76  # no orientation
index_Spine_Shoulder = 80
index_Tip_Left = 84  # no orientation
index_Thumb_Left = 88  # no orientation
index_Tip_Right = 92  # no orientation
index_Thumb_Right = 96  # no orientation

In [ ]:
# Low-pass filter function
# Input: np array of shape (n_frames, 1)
def filtering(x):
    # Define filter parameters
    filtCutOff = 1
    sample = 30
    # Design a Butterworth low-pass filter
    b, a = butter(3, (2 * filtCutOff) / sample, "low")
    # Apply the filter to the input signal x using filtfilt
    media = filtfilt(b, a, x)
    return media

In [ ]:
# Filter 3-dimensional data array
# Input: np array of shape (n_frames, 3)
def filter_3D(x):
  # Create empty np array the same size as input array
  media = np.empty([x.shape[0], x.shape[1]])
  # Filter data from each axis
  media[:,0] = filtering(x[:,0])
  media[:,1] = filtering(x[:,1])
  media[:,2] = filtering(x[:,2])
  return media

In [ ]:
# Read and normalize labelsfrom excel file
# Input: path to .xlsx file
def normalize_labels(path_to_xlsx):
    # Read the .xlsx file into a DataFrame
    df = pd.read_excel(path_to_xlsx)
    # Storing the raw scores
    raw_scores = df.drop("Subject ID", axis=1)
    # Convert the DataFrame to a NumPy array
    raw_scores_array = raw_scores.to_numpy()
    #print(raw_scores_array)
    raw_scores_array = raw_scores_array.astype('float64')

    # Normalise labels: Divide the first 5 values by 50, second 5 by 15, last 5 by 25
    raw_scores_array[0, :5] = raw_scores_array[0, :5] / 50.0
    raw_scores_array[0, 5:10] = raw_scores_array[0, 5:10] / 15.0
    raw_scores_array[0, 10:] = raw_scores_array[0, 10:] / 35.0
    #print(raw_scores_array)

    # Select the first, sixth, and eleventh values from the raw_scores_array (Excercise 1)
    exercise_1_score = raw_scores_array[0, [0, 5, 10]]
    # Select the second, seventh, and twelfth values from the raw_scores_array (Excerise 2)
    exercise_2_score = raw_scores_array[0, [1, 6, 11]]
    # Select the third, eight, and thirteenth values from the raw_scores_array (Excerise 3)
    exercise_3_score = raw_scores_array[0, [2, 7, 12]]
    # Exercise 4: Select the fourth, ninth, and fourteenth values from the raw_scores (Excercise 4)
    exercise_4_score = raw_scores_array[0, [3, 8, 13]]
    # Exercise 5: Select the fifth, tenth, and fifteenth values from the raw_scores (Excercise 5)
    exercise_5_score = raw_scores_array[0, [4, 9, 14]]

    # Return normalised scores in list
    return [exercise_1_score, exercise_2_score, exercise_3_score, exercise_4_score, exercise_5_score]

In [ ]:
## Added back: E_ID8 (Es1), P_ID3 (Es1), P_ID5 (Es1), S_ID6 (Es5)
## Removed: E_ID17 (No labels)
# Patient IDs for KIMORE dataset
ids = ['E_ID1', 'E_ID2', 'E_ID3', 'E_ID4', 'E_ID5', 'E_ID6', 'E_ID7', 'E_ID8', 'E_ID9', 'E_ID10', 'E_ID11', 'E_ID12', 'E_ID13',
       'E_ID14', 'E_ID15', 'E_ID16', 'NE_ID1', 'NE_ID2', 'NE_ID3', 'NE_ID4', 'NE_ID5', 'NE_ID6', 'NE_ID7', 'NE_ID8', 'NE_ID9', 'NE_ID10',
       'NE_ID11', 'NE_ID12', 'NE_ID13', 'NE_ID14', 'NE_ID15', 'NE_ID16', 'NE_ID17', 'NE_ID18', 'NE_ID19', 'NE_ID20', 'NE_ID21', 'NE_ID22',
       'NE_ID23', 'NE_ID24', 'NE_ID25', 'NE_ID26', 'NE_ID27', 'B_ID1', 'B_ID2', 'B_ID3', 'B_ID4', 'B_ID5', 'B_ID6', 'B_ID7', 'B_ID8',
       'P_ID1', 'P_ID2', 'P_ID3', 'P_ID4', 'P_ID5', 'P_ID6', 'P_ID7', 'P_ID8', 'P_ID9', 'P_ID10','P_ID11', 'P_ID12', 'P_ID13',
       'P_ID14', 'P_ID15', 'P_ID16', 'S_ID1', 'S_ID2', 'S_ID3', 'S_ID4', 'S_ID5', 'S_ID6', 'S_ID7', 'S_ID8', 'S_ID9', 'S_ID10']
# Reduce ID list (use for testing)
red_ids = ['E_ID1', 'NE_ID1', 'B_ID1', 'P_ID1', 'S_ID1']
# List of exercises
ess = ['Es1', 'Es2', 'Es3', 'Es4', 'Es5']

In [ ]:
# Partial random method
# Create split dictionary (contains lists of IDs)

# Set fraction for train/total data
train_test_split = 0.8
ids_size = len(ids)
ind_split = int(train_test_split * ids_size)

# Split list into different sections
np.random.shuffle(ids)
ids_train = ids[:ind_split]
ids_val = ids[ind_split:]

# List all combinations with patient IDs and exercises
id_es_train = []
for id in ids_train:
  for es in ess:
    id_es_train.append(id+'_'+es)
id_es_val = []
for id in ids_val:
  for es in ess:
    id_es_val.append(id+'_'+es)

# Construct dictionary
#split_dict = {'x_train': id_es_train, 'x_val': id_es_val}
split_dict = {'xsub_train': id_es_train, 'xsub_val': id_es_val}
split_dict

In [ ]:
# Fully random method
# Create split dictionary (contains lists of IDs)

# List all combinations with patient IDs and exercises
id_es = []
for id in ids:
  for es in ess:
    id_es.append(id+'_'+es)

# Set fraction for train/total data
train_test_split = 0.8
id_es_size = len(id_es)
ind_split = int(train_test_split * id_es_size)

# Split list into different sections
np.random.shuffle(id_es)
id_es_train = id_es[:ind_split]
id_es_val = id_es[ind_split:]

# Construct dictionary
#split_dict = {'x_train': id_es_train, 'x_val': id_es_val}
split_dict = {'xsub_train': id_es_train, 'xsub_val': id_es_val}
split_dict

{'xsub_train': ['E_ID14_Es3',
  'B_ID3_Es5',
  'NE_ID27_Es2',
  'NE_ID13_Es4',
  'S_ID9_Es1',
  'E_ID5_Es2',
  'NE_ID20_Es1',
  'NE_ID15_Es1',
  'S_ID3_Es3',
  'NE_ID15_Es3',
  'P_ID6_Es3',
  'NE_ID23_Es4',
  'NE_ID11_Es3',
  'NE_ID22_Es4',
  'S_ID3_Es4',
  'P_ID12_Es5',
  'S_ID9_Es3',
  'P_ID1_Es2',
  'P_ID10_Es5',
  'E_ID2_Es5',
  'B_ID4_Es3',
  'NE_ID8_Es5',
  'NE_ID13_Es5',
  'NE_ID26_Es2',
  'E_ID4_Es2',
  'P_ID15_Es5',
  'NE_ID8_Es1',
  'E_ID6_Es5',
  'P_ID16_Es4',
  'NE_ID5_Es3',
  'NE_ID7_Es5',
  'B_ID7_Es1',
  'S_ID2_Es2',
  'E_ID5_Es5',
  'P_ID8_Es5',
  'E_ID12_Es3',
  'P_ID9_Es5',
  'E_ID16_Es5',
  'E_ID2_Es1',
  'E_ID16_Es3',
  'P_ID15_Es4',
  'NE_ID21_Es3',
  'P_ID6_Es2',
  'S_ID6_Es1',
  'P_ID5_Es1',
  'P_ID2_Es1',
  'B_ID4_Es1',
  'B_ID8_Es5',
  'E_ID4_Es5',
  'E_ID14_Es2',
  'P_ID14_Es5',
  'P_ID4_Es4',
  'E_ID9_Es1',
  'E_ID10_Es3',
  'S_ID8_Es2',
  'B_ID8_Es2',
  'E_ID12_Es1',
  'NE_ID4_Es4',
  'NE_ID27_Es1',
  'S_ID3_Es5',
  'B_ID5_Es3',
  'B_ID4_Es4',
  'P_ID1_Es3',

In [ ]:
# Create annotations list (contains dictionaries with joint position data, labels)
annots_list = []

# Constants
n_persons = 1
n_keypoints = 25
n_dims = 3
n_labels = 3

In [ ]:
# Add filtered data to annotations list
# Path to data
dir_path = '/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/'

# Iterate through each Patient ID
for id in ids:

  # Find single Clinical assessment xlsx file in Label folder
  # Get labels for all 5 exercises (for single patient ID)
  for label in glob.glob(dir_path+id+'/Es1/Label/Clinical*'):
    #print(label)
    es_labels = normalize_labels(label)

  # Iterate through each exercise
  for es in ess:
    print ('Patient, Exercise:', id, es)
    # Find single joint position csv file in Raw folder
    for joint_pos in glob.glob(dir_path+id+'/'+es+'/Raw/JointPosition*'):
      print(joint_pos)
      joint_positions = pd.read_csv(joint_pos, header=None)
      # Drop extra column
      if joint_positions.shape[1] > 100:
        joint_positions = joint_positions.drop(100, axis=1)
      positions = joint_positions.to_numpy()

      # Separate into individual joint positions, low-pass filter data on three axes
      Hip_R = filter_3D(positions[:, index_Hip_Right : index_Hip_Right + n_dims])
      Knee_R = filter_3D(positions[:, index_Knee_Right : index_Knee_Right + n_dims])
      Ankle_R = filter_3D(positions[:, index_Ankle_Right : index_Ankle_Right + n_dims])
      Foot_R = filter_3D(positions[:, index_Foot_Right : index_Foot_Right + n_dims])
      Hip_L = filter_3D(positions[:, index_Hip_Left : index_Hip_Left + n_dims])
      Knee_L = filter_3D(positions[:, index_Knee_Left : index_Knee_Left + n_dims])
      Ankle_L = filter_3D(positions[:, index_Ankle_Left : index_Ankle_Left + n_dims])
      Foot_L = filter_3D(positions[:, index_Foot_Left : index_Foot_Left + n_dims])
      Spine_B = filter_3D(positions[:, index_Spine_Base : index_Spine_Base + n_dims])
      Spine_M = filter_3D(positions[:, index_Spine_Mid : index_Spine_Mid + n_dims])
      Head_C = filter_3D(positions[:, index_Head : index_Head + n_dims])
      Neck = filter_3D(positions[:, index_Neck : index_Neck + n_dims])
      Spine_Sh = filter_3D(positions[:, index_Spine_Shoulder : index_Spine_Shoulder + n_dims])
      Shoulder_R = filter_3D(positions[:, index_Shoulder_Right : index_Shoulder_Right + n_dims])
      Shoulder_L = filter_3D(positions[:, index_Shoulder_Left : index_Shoulder_Left + n_dims])
      Elbow_R = filter_3D(positions[:, index_Elbow_Right : index_Elbow_Right + n_dims])
      Wrist_R = filter_3D(positions[:, index_Wrist_Right : index_Wrist_Right + n_dims])
      Hand_R = filter_3D(positions[:, index_Hand_Right : index_Hand_Right + n_dims])
      Elbow_L = filter_3D(positions[:, index_Elbow_Left : index_Elbow_Left + n_dims])
      Wrist_L = filter_3D(positions[:, index_Wrist_Left : index_Wrist_Left + n_dims])
      Hand_L = filter_3D(positions[:, index_Hand_Left : index_Hand_Left + n_dims])
      Tip_L = filter_3D(positions[:, index_Tip_Left : index_Tip_Left + n_dims])
      Thumb_L = filter_3D(positions[:, index_Thumb_Left : index_Thumb_Left + n_dims])
      Tip_R = filter_3D(positions[:, index_Tip_Right : index_Tip_Right + n_dims])
      Thumb_R = filter_3D(positions[:, index_Thumb_Right : index_Thumb_Right + n_dims])

      # Create dictionary specific patient and exercise combination
      n_frames = positions.shape[0]
      ind = ess.index(es) # Index of exercise (0,1,2,3,4)
      scores = es_labels[ind]
      # Dictionary declaration
      new_dict = {'frame_dir': id+'_'+es,
                  'label': scores,
                  'total_frames': n_frames,
                  'keypoint': np.empty([1, n_frames, n_keypoints, n_dims])}
      # Add data to keypoint arrays
      for i in range(n_frames):
        new_dict['keypoint'][0][i] = np.array([Spine_B[i], Spine_M[i], Neck[i], Head_C[i],
                                         Shoulder_L[i], Elbow_L[i], Wrist_L[i], Hand_L[i],
                                         Shoulder_R[i], Elbow_R[i], Wrist_R[i], Hand_R[i],
                                         Hip_L[i], Knee_L[i], Ankle_L[i], Foot_L[i],
                                         Hip_R[i], Knee_R[i], Ankle_R[i], Foot_R[i],
                                         Spine_Sh[i], Tip_L[i], Thumb_L[i], Tip_R[i], Thumb_R[i]])
      annots_list.append(new_dict)
    # End joint position csv loop (only runs once)
  # End ex loop
# End id loop

In [ ]:
# Add unfiltered data to annotations list
# Path to data
dir_path = '/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/'

# Iterate through each Patient ID
for id in ids:

  # Find single Clinical assessment xlsx file in Label folder
  # Get labels for all 5 exercises (for single patient ID)
  for label in glob.glob(dir_path+id+'/Es1/Label/Clinical*'):
    #print(label)
    es_labels = normalize_labels(label)

  # Iterate through each exercise
  for es in ess:
    print ('Patient, Exercise:', id, es)
    # Find single joint position csv file in Raw folder
    for joint_pos in glob.glob(dir_path+id+'/'+es+'/Raw/JointPosition*'):
      print(joint_pos)
      joint_positions = pd.read_csv(joint_pos, header=None)
      # Drop extra column
      if joint_positions.shape[1] > 100:
        joint_positions = joint_positions.drop(100, axis=1)
      positions = joint_positions.to_numpy()

      # Separate into individual joint positions, low-pass filter data on three axes
      Hip_R = positions[:, index_Hip_Right : index_Hip_Right + n_dims]
      Knee_R = positions[:, index_Knee_Right : index_Knee_Right + n_dims]
      Ankle_R = positions[:, index_Ankle_Right : index_Ankle_Right + n_dims]
      Foot_R = positions[:, index_Foot_Right : index_Foot_Right + n_dims]
      Hip_L = positions[:, index_Hip_Left : index_Hip_Left + n_dims]
      Knee_L = positions[:, index_Knee_Left : index_Knee_Left + n_dims]
      Ankle_L = positions[:, index_Ankle_Left : index_Ankle_Left + n_dims]
      Foot_L = positions[:, index_Foot_Left : index_Foot_Left + n_dims]
      Spine_B = positions[:, index_Spine_Base : index_Spine_Base + n_dims]
      Spine_M = positions[:, index_Spine_Mid : index_Spine_Mid + n_dims]
      Head_C = positions[:, index_Head : index_Head + n_dims]
      Neck = positions[:, index_Neck : index_Neck + n_dims]
      Spine_Sh = positions[:, index_Spine_Shoulder : index_Spine_Shoulder + n_dims]
      Shoulder_R = positions[:, index_Shoulder_Right : index_Shoulder_Right + n_dims]
      Shoulder_L = positions[:, index_Shoulder_Left : index_Shoulder_Left + n_dims]
      Elbow_R = positions[:, index_Elbow_Right : index_Elbow_Right + n_dims]
      Wrist_R = positions[:, index_Wrist_Right : index_Wrist_Right + n_dims]
      Hand_R = positions[:, index_Hand_Right : index_Hand_Right + n_dims]
      Elbow_L = positions[:, index_Elbow_Left : index_Elbow_Left + n_dims]
      Wrist_L = positions[:, index_Wrist_Left : index_Wrist_Left + n_dims]
      Hand_L = positions[:, index_Hand_Left : index_Hand_Left + n_dims]
      Tip_L = positions[:, index_Tip_Left : index_Tip_Left + n_dims]
      Thumb_L = positions[:, index_Thumb_Left : index_Thumb_Left + n_dims]
      Tip_R = positions[:, index_Tip_Right : index_Tip_Right + n_dims]
      Thumb_R = positions[:, index_Thumb_Right : index_Thumb_Right + n_dims]

      # Create dictionary specific patient and exercise combination
      n_frames = positions.shape[0]
      ind = ess.index(es) # Index of exercise (0,1,2,3,4)
      scores = es_labels[ind]
      # Dictionary declaration
      new_dict = {'frame_dir': id+'_'+es,
                  'label': scores,
                  'total_frames': n_frames,
                  'keypoint': np.empty([1, n_frames, n_keypoints, n_dims])}
      # Add data to keypoint arrays
      for i in range(n_frames):
        new_dict['keypoint'][0][i] = np.array([Spine_B[i], Spine_M[i], Neck[i], Head_C[i],
                                         Shoulder_L[i], Elbow_L[i], Wrist_L[i], Hand_L[i],
                                         Shoulder_R[i], Elbow_R[i], Wrist_R[i], Hand_R[i],
                                         Hip_L[i], Knee_L[i], Ankle_L[i], Foot_L[i],
                                         Hip_R[i], Knee_R[i], Ankle_R[i], Foot_R[i],
                                         Spine_Sh[i], Tip_L[i], Thumb_L[i], Tip_R[i], Thumb_R[i]])
      annots_list.append(new_dict)
    # End joint position csv loop (only runs once)
  # End ex loop
# End id loop

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Patient, Exercise: E_ID1 Es1
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID1/Es1/Raw/JointPosition011214_103748.csv
Patient, Exercise: E_ID1 Es2
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID1/Es2/Raw/JointPosition011214_103845.csv
Patient, Exercise: E_ID1 Es3
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID1/Es3/Raw/JointPosition011214_103956.csv
Patient, Exercise: E_ID1 Es4
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID1/Es4/Raw/JointPosition011214_104108.csv
Patient, Exercise: E_ID1 Es5
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID1/Es5/Raw/JointPosition011214_104211.csv


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Patient, Exercise: E_ID2 Es1
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID2/Es1/Raw/JointPosition011214_120912.csv
Patient, Exercise: E_ID2 Es2
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID2/Es2/Raw/JointPosition011214_120957.csv
Patient, Exercise: E_ID2 Es3
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID2/Es3/Raw/JointPosition011214_121037.csv
Patient, Exercise: E_ID2 Es4
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID2/Es4/Raw/JointPosition011214_121119.csv
Patient, Exercise: E_ID2 Es5
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/E_ID2/Es5/Raw/JointPosition011214_121234.csv


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


...

Patient, Exercise: S_ID8 Es1
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID8/Es1/Raw/JointPosition271114_113612.csv
Patient, Exercise: S_ID8 Es2
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID8/Es2/Raw/JointPosition271114_113727.csv
Patient, Exercise: S_ID8 Es3
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID8/Es3/Raw/JointPosition271114_113924.csv
Patient, Exercise: S_ID8 Es4
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID8/Es4/Raw/JointPosition271114_114058.csv
Patient, Exercise: S_ID8 Es5
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID8/Es5/Raw/JointPosition271114_114206.csv


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Patient, Exercise: S_ID9 Es1
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID9/Es1/Raw/JointPosition271114_120405.csv
Patient, Exercise: S_ID9 Es2
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID9/Es2/Raw/JointPosition271114_120505.csv
Patient, Exercise: S_ID9 Es3
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID9/Es3/Raw/JointPosition271114_120547.csv
Patient, Exercise: S_ID9 Es4
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID9/Es4/Raw/JointPosition271114_120626.csv
Patient, Exercise: S_ID9 Es5
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID9/Es5/Raw/JointPosition271114_120741.csv


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Patient, Exercise: S_ID10 Es1
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID10/Es1/Raw/JointPosition151214_132148.csv
Patient, Exercise: S_ID10 Es2
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID10/Es2/Raw/JointPosition151214_132336.csv
Patient, Exercise: S_ID10 Es3
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID10/Es3/Raw/JointPosition151214_132544.csv
Patient, Exercise: S_ID10 Es4
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID10/Es4/Raw/JointPosition151214_132701.csv
Patient, Exercise: S_ID10 Es5
/content/gdrive/MyDrive/UofT/BME1570/BME1570_Team2/Kimore/S_ID10/Es5/Raw/JointPosition151214_132813.csv


In [ ]:
# Put dictionary together
skeleton_3D = {'split': split_dict, 'annotations': annots_list}

# Write dictionaries to pkl file
with open('skeleton_3D.pkl', 'wb') as file:
  # Pickle the object and write it to the file
  pickle.dump(skeleton_3D, file)

In [ ]:
# View pkl file
skeleton_3D

{'split': {'xsub_train': ['E_ID14_Es3',
   'B_ID3_Es5',
   'NE_ID27_Es2',
   'NE_ID13_Es4',
   'S_ID9_Es1',
   'E_ID5_Es2',
   'NE_ID20_Es1',
   'NE_ID15_Es1',
   'S_ID3_Es3',
   'NE_ID15_Es3',
   'P_ID6_Es3',
   'NE_ID23_Es4',
   'NE_ID11_Es3',
   'NE_ID22_Es4',
   'S_ID3_Es4',
   'P_ID12_Es5',
   'S_ID9_Es3',
   'P_ID1_Es2',
   'P_ID10_Es5',
   'E_ID2_Es5',
   'B_ID4_Es3',
   'NE_ID8_Es5',
   'NE_ID13_Es5',
   'NE_ID26_Es2',
   'E_ID4_Es2',
   'P_ID15_Es5',
   'NE_ID8_Es1',
   'E_ID6_Es5',
   'P_ID16_Es4',
   'NE_ID5_Es3',
   'NE_ID7_Es5',
   'B_ID7_Es1',
   'S_ID2_Es2',
   'E_ID5_Es5',
   'P_ID8_Es5',
   'E_ID12_Es3',
   'P_ID9_Es5',
   'E_ID16_Es5',
   'E_ID2_Es1',
   'E_ID16_Es3',
   'P_ID15_Es4',
   'NE_ID21_Es3',
   'P_ID6_Es2',
   'S_ID6_Es1',
   'P_ID5_Es1',
   'P_ID2_Es1',
   'B_ID4_Es1',
   'B_ID8_Es5',
   'E_ID4_Es5',
   'E_ID14_Es2',
   'P_ID14_Es5',
   'P_ID4_Es4',
   'E_ID9_Es1',
   'E_ID10_Es3',
   'S_ID8_Es2',
   'B_ID8_Es2',
   'E_ID12_Es1',
   'NE_ID4_Es4',
   'NE_ID

In [ ]:
for annot_dict in skeleton_3D['annotations']:
  if np.isnan(annot_dict['label'][0]):
    print(annot_dict['frame_dir'])
  #endif
#endfor

In [ ]:
# Read pkl file
pkl_path = '/content/skeleton_3D.pkl'

with open(pkl_path, 'rb') as f:
  skeletal_data = pickle.load(f)
skeletal_data